### Example codes from snapvx
Example 1. We first look at a simple example with two nodes and one edge between them. The two nodes have different objectives, both in $\mathbb R^1$, and the edge objective penalizes the square of the difference of the variables.

In [2]:
from snapvx import *
#Create new graph
gvx = TGraphVX()

#Use CVXPY syntax to define a problem
x1 = Variable(1, name='x1')
obj = square(x1)
#Add Node 1 with the given objective, with the constraint that x1 <= 10
gvx.AddNode(1, Objective=obj, Constraints=[x1 <= 10])

#Similarly, add Node 2 with objective |x2 + 3|
x2 = Variable(1, name='x2')
obj2 = abs(x2 + 3)
gvx.AddNode(2, obj2, [])

#Add an edge between the two nodes, 
#Define an objective, constraints using CVXPY syntax
gvx.AddEdge(1, 2, Objective=square(norm(x1 - x2)), Constraints=[])

gvx.Solve() #Solve the problem
gvx.PrintSolution() #Print entire solution on a node-by-node basis
print "x1 = ", x1.value, "; x2 = ", x2.value #Print the solutions of individual variables

Status: Optimal
Total Objective: 2.500335
Node 1:
  x1 [-0.51706729]
Node 2:
  x2 [-1.02366535]
x1 =  -0.517067292938 ; x2 =  -1.02366535108


Example 2: Random Graph. Next, we look at a slightly larger example. We build a random graph (using GenRndGnm), set each node objective to the square loss from a random point in $\mathbb R^{10}$, and use laplacian regularization on the edges.

In [14]:
from snapvx import *
import numpy

#Helper function to define edge objectives
#Takes in two nodes, returns a problem defined with the variables from those nodes
def laplace_reg(src, dst, data):
  return (sum_squares(src['x'] - dst['x']), [])

#Generate random graph, using SNAP syntax
numpy.random.seed(1)
num_nodes = 10
num_edges = 30
n = 10
snapGraph = GenRndGnm(PUNGraph, num_nodes, num_edges)
gvx = TGraphVX(snapGraph)

#For each node, add an objective (using random data)
for i in range(num_nodes):
  x = Variable(n,name='x') #Each node has its own variable named 'x'
  a = numpy.random.randn(n)
  gvx.SetNodeObjective(i, square(norm(x-a)))

#Set all edge objectives at once (Laplacian Regularization)
gvx.AddEdgeObjectives(laplace_reg)

#Solve in verbose mode (using ADMM)
gvx.Solve()
gvx.PrintSolution()

gvx.Solve(UseADMM=False) #Solve serially (no ADMM)
gvx.PrintSolution()

#Print only the solution for 'x' at Node 1
print "Solution at Node 1 is", gvx.GetNodeValue(1,'x')

Status: Optimal
Total Objective: 58.367631
Node 0:
  x [ 0.26654486 -0.33153663 -0.26586178 -0.13822058  0.39428357 -0.81025355  0.32399164  0.19578661  0.33746554  0.34631966]
Node 1:
  x [ 0.18306269 -0.42816156 -0.20520863 -0.07686966  0.42376548 -0.63761584  0.08692271  0.16481059  0.34767127  0.41473059]
Node 2:
  x [-0.19086186 -0.00124975 -0.0013708   0.06803922  0.41479151 -0.54048413  0.02065183  0.04473737  0.25022908  0.4729811 ]
Node 3:
  x [-0.07588868 -0.26492133 -0.25352697 -0.08461505  0.2065418  -0.49406509 -0.06724384  0.30152482  0.49299726  0.484503  ]
Node 4:
  x [-0.00061248 -0.30543156 -0.27286991  0.17645171  0.27753634 -0.57111417  0.1307811   0.56799756  0.37111355  0.41903049]
Node 5:
  x [ 0.05174711 -0.28433417 -0.33110526 -0.04504249  0.25776575 -0.45854966  0.19730495  0.36858396  0.34395258  0.49498263]
Node 6:
  x [-0.08502476  0.02836057 -0.06052553 -0.04815531  0.36182207 -0.44462671  0.28200317  0.57590037  0.70465035  0.06207409]
Node 7:
  x [-0.122

Example 3: Bulk Loading. Finally, we look at how we can bulk load the relevant data from a text file

In [6]:
from snapvx import *

#Helper function for node objective
#Takes in a row from the CSV file, returns an optimization problem
def node_obj(data):
  x = Variable(1,name='x')
  return norm(x - float(data[0]))

#Helper function for edge objective
def laplace_reg(src, dst, data):
  return sum_squares(src['x'] - dst['x'])

#Load in Edge List to build graph with default node/edge objectives
gvx = LoadEdgeList('BulkLoadEdges.edges')

#Bulk Load node objectives:
#Takes one row of the CSV, uses that as input to node_obj
#There is also an (optional) input of specifying which nodes each row of the CSV refers to
gvx.AddNodeObjectives('BulkLoadData.csv', node_obj)

#Bulk load edge objectives for all edges
gvx.AddEdgeObjectives(laplace_reg)

gvx.Solve()
gvx.PrintSolution()

Status: Optimal
Total Objective: 2.750145
Node 1:
  x [ 0.34398022]
Node 2:
  x [ 0.85601343]
